### Background:

As a talent sourcing and management company, we are interested in finding talented individuals for sourcing these candidates to technology companies. Finding talented candidates is not easy, for several reasons. The first reason is one needs to understand what the role is very well to fill in that spot, this requires understanding the client’s needs and what they are looking for in a potential candidate. The second reason is one needs to understand what makes a candidate shine for the role we are in search for. Third, where to find talented individuals is another challenge.

The nature of our job requires a lot of human labor and is full of manual operations. Towards automating this process we want to build a better approach that could save us time and finally help us spot potential candidates that could fit the roles we are in search for. Moreover, going beyond that for a specific role we want to fill in we are interested in developing a machine learning powered pipeline that could spot talented individuals, and rank them based on their fitness.

We are right now semi-automatically sourcing a few candidates, therefore the sourcing part is not a concern at this time but we expect to first determine best matching candidates based on how fit these candidates are for a given role. We generally make these searches based on some keywords such as “full-stack software engineer”, “engineering manager” or “aspiring human resources” based on the role we are trying to fill in. These keywords might change, and you can expect that specific keywords will be provided to you.

Assuming that we were able to list and rank fitting candidates, we then employ a review procedure, as each candidate needs to be reviewed and then determined how good a fit they are through manual inspection. This procedure is done manually and at the end of this manual review, we might choose not the first fitting candidate in the list but maybe the 7th candidate in the list. If that happens, we are interested in being able to re-rank the previous list based on this information. This supervisory signal is going to be supplied by starring the 7th candidate in the list. Starring one candidate actually sets this candidate as an ideal candidate for the given role. Then, we expect the list to be re-ranked each time a candidate is starred.

### Data Description:

The data comes from our sourcing efforts. We removed any field that could directly reveal personal details and gave a unique identifier for each candidate.

### Attributes:

id : unique identifier for candidate (numeric)

job_title : job title for candidate (text)

location : geographical location for candidate (text)

connections: number of connections candidate has, 500+ means over 500 (text)

### Output (desired target):

fit - how fit the candidate is for the role? (numeric, probability between 0-1)

## Keywords: “Aspiring human resources” or “seeking human resources”


### Goal(s):

Predict how fit the candidate is based on their available information (variable fit)

### Success Metric(s):

Rank candidates based on a fitness score.

Re-rank candidates when a candidate is starred.

### Bonus(es):

We are interested in a robust algorithm, tell us how your solution works and show us how your ranking gets better with each starring action.

How can we filter out candidates which in the first place should not be in this list?

Can we determine a cut-off point that would work for other roles without losing high potential candidates?

Do you have any ideas that we should explore so that we can even automate this procedure to prevent human bias?


In [1]:
# import modules
import random
import os
import warnings
# ignore warnings
warnings.filterwarnings('ignore')

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

# sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem import WordNetLemmatizer
# scipy
from scipy.spatial.distance import cosine
# gensim
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors, Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec

%matplotlib inline 



In [2]:
# load the dataset
df = pd.read_csv('potential-talents.csv')

In [3]:
df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          104 non-null    int64  
 1   job_title   104 non-null    object 
 2   location    104 non-null    object 
 3   connection  104 non-null    object 
 4   fit         0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.2+ KB


In [5]:
df.drop(['fit','id'], axis = 1, inplace = True)             


In [6]:
df = df.drop_duplicates(ignore_index=True) 

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   job_title   53 non-null     object
 1   location    53 non-null     object
 2   connection  53 non-null     object
dtypes: object(3)
memory usage: 1.4+ KB


In [8]:
df.head()

,job_title,location,connection
0,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85
1,Native English Teacher at EPIK (English Progra...,Kanada,500+
2,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44
3,People Development Coordinator at Ryan,"Denton, Texas",500+
4,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+


## Create a function for preprocessing with "spaCy" NLP library

In [9]:
def is_token_allowed(token):
#    '''
#         Only allow valid tokens which are not stop words
#         and punctuation symbols.
#     '''
     if (not token or not token.text.strip() or
         token.is_stop or token.is_punct):
         return False
     return True

def preprocess_token(token):
     # Reduce token to its lowercase lemma form
     return token.lemma_.strip().lower()

#complete_filtered_tokens = [preprocess_token(token)
#for token in df_doc if is_token_allowed(token)]

In [10]:
# Create a new column as a copy of job_title, to be updated with preprocessing
df['nlp_job_title'] = df ['job_title']

In [11]:
for i in range (0, len(df)):
    df_doc = nlp(df.job_title[i])
    preproc_tokens = [preprocess_token(token)for token in df_doc if is_token_allowed(token)]
    df['nlp_job_title'][i] = " ".join(preproc_tokens)


In [12]:
complete_filtered_tokens = [preprocess_token(token)
for token in df_doc if is_token_allowed(token)]
complete_filtered_tokens

['director', 'administration', 'excellence', 'logging']

In [13]:
df.head(10)

,job_title,location,connection,nlp_job_title
0,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,2019 c.t. bauer college business graduate magn...
1,Native English Teacher at EPIK (English Progra...,Kanada,500+,native english teacher epik english program korea
2,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,aspiring human resources professional
3,People Development Coordinator at Ryan,"Denton, Texas",500+,people development coordinator ryan
4,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,advisory board member celal bayar university
5,Aspiring Human Resources Specialist,Greater New York City Area,1,aspiring human resources specialist
6,Student at Humber College and Aspiring Human R...,Kanada,61,student humber college aspiring human resource...
7,HR Senior Specialist,San Francisco Bay Area,500+,hr senior specialist
8,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,seek human resources hris generalist positions
9,Student at Chapman University,"Lake Forest, California",2,student chapman university


## Create a similar procedure for preprocessing with NLTK

In [14]:
#Create another job title column for comparing preprocessing using the "nltk" NLP library
df['cleaned_job_title'] = df['job_title']

In [15]:
# Removing unnecssary characters like pipes, comma, dot etc. for cleaning the data.
df['cleaned_job_title'] = df['cleaned_job_title'].str.replace(r"[^a-zA-Z0-9\' ']", "")
df['cleaned_job_title'] = df['cleaned_job_title'].str.lower()
df = df.reset_index(drop = True)

# Remove stop words and lemmatize words in cleaned job title
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

for i in range (0, len(df)):
    word_tokens = word_tokenize(df['cleaned_job_title'][i])
    filtered_sentence = [w for w in word_tokens if w not in stop_words]
    lemmatized_sentence = []
    for word in filtered_sentence:
        lemmatized_sentence.append(lemmatizer.lemmatize(word))
    df['cleaned_job_title'][i] = TreebankWordDetokenizer().detokenize(lemmatized_sentence)

df.head(10)

,job_title,location,connection,nlp_job_title,cleaned_job_title
0,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,2019 c.t. bauer college business graduate magn...,2019 ct bauer college business graduate magna ...
1,Native English Teacher at EPIK (English Progra...,Kanada,500+,native english teacher epik english program korea,native english teacher epik english program korea
2,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,aspiring human resources professional,aspiring human resource professional
3,People Development Coordinator at Ryan,"Denton, Texas",500+,people development coordinator ryan,people development coordinator ryan
4,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,advisory board member celal bayar university,advisory board member celal bayar university
5,Aspiring Human Resources Specialist,Greater New York City Area,1,aspiring human resources specialist,aspiring human resource specialist
6,Student at Humber College and Aspiring Human R...,Kanada,61,student humber college aspiring human resource...,student humber college aspiring human resource...
7,HR Senior Specialist,San Francisco Bay Area,500+,hr senior specialist,hr senior specialist
8,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,seek human resources hris generalist positions,seeking human resource hris generalist position
9,Student at Chapman University,"Lake Forest, California",2,student chapman university,student chapman university


## It appears the preprocessing produced the same result with both methods, except spaCy is preserving the periods ( c.t. bauer vs ct bauer, inc. vs inc). When converting to a list for vectorization, the number of features is 183 vs 180.

## Let's check the one that keeps more features and try to identify if the "periods" are causing there to be 3 more features 

In [16]:
# Convert job_title column into a list
nlp_title_list = list(df['nlp_job_title'])                    
cleaned_title_list = list(df['cleaned_job_title'])                    

# Vectorize job_title_list
vectorizer = TfidfVectorizer()                                 
# Specify which list to vectorize
vectorizer.fit(nlp_title_list)                    
X = vectorizer.transform(nlp_title_list)    

#vectorizer.fit(cleaned_title_list)                    
#XX = vectorizer.transform(cleaned_title_list)    


In [17]:
nlp_title_list

['2019 c.t. bauer college business graduate magna cum laude aspire human resources professional',
 'native english teacher epik english program korea',
 'aspiring human resources professional',
 'people development coordinator ryan',
 'advisory board member celal bayar university',
 'aspiring human resources specialist',
 'student humber college aspiring human resources generalist',
 'hr senior specialist',
 'seek human resources hris generalist positions',
 'student chapman university',
 'svp chro marketing communications csr officer | engie | houston | woodlands | energy | gphr | sphr',
 'human resources coordinator intercontinental buckhead atlanta',
 'aspiring human resources management student seek internship',
 'seek human resources opportunities',
 'experienced retail manager aspire human resources professional',
 'human resources staffing recruiting professional',
 'human resources specialist luxottica',
 'director human resources north america groupe beneteau',
 'retired army 

In [18]:
# Get feature names in all the documents
feature_names = vectorizer.get_feature_names()                  
print("Number of unique features: ", len(feature_names))        

Number of unique features:  183


In [19]:
# Convert job titles into arrays
tfidf_vector = X.toarray()                                  
print("Shape of Tfidf vector: ", tfidf_vector.shape)        

Shape of Tfidf vector:  (53, 183)


## Cosine Similarity

In [20]:
feature_names

['2019',
 '2020',
 '2621',
 '408',
 '709',
 'administration',
 'administrative',
 'admissions',
 'advisory',
 'america',
 'analyst',
 'analytics',
 'army',
 'arts',
 'aspire',
 'aspiring',
 'assistant',
 'atlanta',
 'bachelor',
 'bauer',
 'bayar',
 'beach',
 'benefits',
 'beneteau',
 'biology',
 'board',
 'brand',
 'buckhead',
 'business',
 'care',
 'celal',
 'center',
 'chapman',
 'chro',
 'college',
 'communications',
 'community',
 'compensation',
 'conflict',
 'coordinator',
 'create',
 'csr',
 'cum',
 'customer',
 'datum',
 'delphi',
 'development',
 'director',
 'employment',
 'endemol',
 'energetic',
 'energy',
 'engaging',
 'engie',
 'engineer',
 'english',
 'entry',
 'environment',
 'environmental',
 'epik',
 'excellence',
 'executive',
 'experienced',
 'ey',
 'focus',
 'generalist',
 'gis',
 'gphr',
 'graduate',
 'groupe',
 'guard',
 'hardware',
 'heil',
 'help',
 'houston',
 'hr',
 'hris',
 'human',
 'humber',
 'illinois',
 'inc',
 'inclusive',
 'indiana',
 'information',
 '

In [21]:
tfidf_vector[0]

array([0.34239355, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.28714724,
       0.        , 0.        , 0.        , 0.        , 0.34239355,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.25483022, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.31007653,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.34239355, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.31007653, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

## Use the first suggested search phrase to compare against the vector

In [22]:
# Searched phrase
phrase = 'Aspiring human resources'

In [23]:
# Define cosine similarity function (cosine gives cosine distance)
def cosine_similarity(vector_1, vector_2):
    cos_sim = 1 - cosine(vector_1, vector_2)        
    return cos_sim

In [24]:
# Lemmatize searched phrase
lemmatized_phrase = []
tokenized_phrase = word_tokenize(phrase.lower())
for word in tokenized_phrase:
    lemmatized_phrase.append(lemmatizer.lemmatize(word))
lemmatized_phrase = TreebankWordDetokenizer().detokenize(lemmatized_phrase)

# Convert search phrase into a vector
X1 = vectorizer.transform([lemmatized_phrase])                    
X1_vector = X1.toarray()
print("Shape of search phrase vector:", X1_vector.shape)

Shape of search phrase vector: (1, 183)


In [25]:
# Calcualte Tfidf cosine similarity and add it to dataframe
sim_score_list = []
for x in range (0, len(df)):
    sim_score_list.append(cosine_similarity(tfidf_vector[x], X1_vector))

df['tfidf_sim_score'] = sim_score_list
df.sort_values(by ='tfidf_sim_score', ascending = False).head(20)

,job_title,location,connection,nlp_job_title,cleaned_job_title,tfidf_sim_score
2,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,aspiring human resources professional,aspiring human resource professional,0.697367
45,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,aspiring human resources professional,aspiring human resource professional,0.697367
5,Aspiring Human Resources Specialist,Greater New York City Area,1,aspiring human resources specialist,aspiring human resource specialist,0.629129
21,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,aspiring human resources manager seek internsh...,aspiring human resource manager seeking intern...,0.504342
12,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+,aspiring human resources management student se...,aspiring human resource management student see...,0.412685
20,Business Management Major and Aspiring Human R...,"Monroe, Louisiana Area",5,business management major aspiring human resou...,business management major aspiring human resou...,0.410509
6,Student at Humber College and Aspiring Human R...,Kanada,61,student humber college aspiring human resource...,student humber college aspiring human resource...,0.373073
30,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,aspiring human resources professional | energe...,aspiring human resource professional energetic...,0.317168
48,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,aspiring human resources manager | graduate 20...,aspiring human resource manager graduating may...,0.295197
24,Aspiring Human Resources Professional | Passio...,"New York, New York",212,aspiring human resources professional | passio...,aspiring human resource professional passionat...,0.246154


In [26]:
# Convert job_title column into a list
#nlp_title_list = list(df['nlp_job_title'])                    
#cleaned_title_list = list(df['cleaned_job_title'])                    

# Vectorize job_title_list
vectorizer = TfidfVectorizer()                                 
# Specify which list to vectorize
#vectorizer.fit(nlp_title_list)                    
#X = vectorizer.transform(nlp_title_list)    

vectorizer.fit(cleaned_title_list)                    
XX = vectorizer.transform(cleaned_title_list)    


In [27]:
cleaned_title_list

['2019 ct bauer college business graduate magna cum laude aspiring human resource professional',
 'native english teacher epik english program korea',
 'aspiring human resource professional',
 'people development coordinator ryan',
 'advisory board member celal bayar university',
 'aspiring human resource specialist',
 'student humber college aspiring human resource generalist',
 'hr senior specialist',
 'seeking human resource hris generalist position',
 'student chapman university',
 'svp chro marketing communication csr officer engie houston woodland energy gphr sphr',
 'human resource coordinator intercontinental buckhead atlanta',
 'aspiring human resource management student seeking internship',
 'seeking human resource opportunity',
 'experienced retail manager aspiring human resource professional',
 'human resource staffing recruiting professional',
 'human resource specialist luxottica',
 'director human resource north america groupe beneteau',
 'retired army national guard rec

In [28]:
# Get feature names in all the documents
feature_names = vectorizer.get_feature_names()                  
print("Number of unique features: ", len(feature_names))        

Number of unique features:  180


In [29]:
# Convert job titles into arrays
tfidf_vector2 = XX.toarray()                                  
print("Shape of Tfidf vector: ", tfidf_vector2.shape)        

Shape of Tfidf vector:  (53, 180)


In [30]:
# Searched phrase
phrase = 'Aspiring human resources'
# Define cosine similarity function (cosine gives cosine distance)
def cosine_similarity(vector_1, vector_2):
    cos_sim = 1 - cosine(vector_1, vector_2)        
    return cos_sim
# Lemmatize searched phrase
lemmatized_phrase = []
tokenized_phrase = word_tokenize(phrase.lower())
for word in tokenized_phrase:
    lemmatized_phrase.append(lemmatizer.lemmatize(word))
lemmatized_phrase = TreebankWordDetokenizer().detokenize(lemmatized_phrase)

# Convert search phrase into a vector
X2 = vectorizer.transform([lemmatized_phrase])                    
X2_vector = X2.toarray()
print("Shape of search phrase vector:", X2_vector.shape)

Shape of search phrase vector: (1, 180)


In [31]:
# Calcualte Tfidf cosine similarity and add it to dataframe
sim_score_list = []
for x in range (0, len(df)):
    sim_score_list.append(cosine_similarity(tfidf_vector2[x], X2_vector))

df['tfidf_sim_score2'] = sim_score_list
df.sort_values(by ='tfidf_sim_score2', ascending = False).head(20)

,job_title,location,connection,nlp_job_title,cleaned_job_title,tfidf_sim_score,tfidf_sim_score2
2,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,aspiring human resources professional,aspiring human resource professional,0.697367,0.774586
45,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,aspiring human resources professional,aspiring human resource professional,0.697367,0.774586
5,Aspiring Human Resources Specialist,Greater New York City Area,1,aspiring human resources specialist,aspiring human resource specialist,0.629129,0.684514
21,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,aspiring human resources manager seek internsh...,aspiring human resource manager seeking intern...,0.504342,0.626699
12,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+,aspiring human resources management student se...,aspiring human resource management student see...,0.412685,0.437541
20,Business Management Major and Aspiring Human R...,"Monroe, Louisiana Area",5,business management major aspiring human resou...,business management major aspiring human resou...,0.410509,0.435239
22,Human Resources Professional,Greater Boston Area,16,human resources professional,human resource professional,0.218047,0.427058
14,Experienced Retail Manager and aspiring Human ...,"Austin, Texas Area",57,experienced retail manager aspire human resour...,experienced retail manager aspiring human reso...,0.091208,0.413698
6,Student at Humber College and Aspiring Human R...,Kanada,61,student humber college aspiring human resource...,student humber college aspiring human resource...,0.373073,0.399351
30,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,aspiring human resources professional | energe...,aspiring human resource professional energetic...,0.317168,0.382502


## While comparing the scores, we can also see some preprocessing differences in lemmatization. For example "[13] opportunities vs opportunity", "[27] aspire vs aspiring" . Some of the scores appear to be significantly different.

# Pre-trained word embedding - various sources

## GloVe (Global Vectors) - Select a publicy available corpus to vectorize and compare with our search phrase. 

In [32]:
# Create temp file and save converted embedding into it
target_file = get_tmpfile('word2vec.6B.300d.txt')
#target_file = get_tmpfile('word2vec.42B.300d.txt')
glove2word2vec('glove.6B.300d.txt', target_file) 
#glove2word2vec('glove.42B.300d.txt', target_file) 


# Load the converted embedding into memory
glove_model = KeyedVectors.load_word2vec_format(target_file)

# Save as binary data
glove_model.save_word2vec_format('word2vec.6B.300d.bin.gz', binary=True)
#glove_model.save_word2vec_format('word2vec.42B.300d.bin.gz', binary=True)

In [33]:
# Define a function to vectorize a sentence's words
def doc_token_vectors(sentence, model, sentence_vector_list, vector_dimensions):
    word_tokens = word_tokenize(sentence)
    filtered_words = [w for w in word_tokens if w in model.key_to_index.keys()]
    
    for j in range(0, len(word_tokens)):
        if word_tokens[j] in filtered_words:
            token_vector = model[word_tokens[j]]
        else:
            token_vector = np.zeros(vector_dimensions)
        sentence_vector_list.append(token_vector)
    
    return sentence_vector_list

In [34]:
# Vectorize job title using GloVe model
glove_vectors = []
for i in range(0, len(df)):
    glove_sentence_vector = []
    doc_token_vectors(df['cleaned_job_title'][i], glove_model, glove_sentence_vector, 300)
    glove_vectors.append(glove_sentence_vector)

In [35]:
# Vectorize searched phrase using GloVe model
glove_search_phrase_vector = []
doc_token_vectors(lemmatized_phrase, glove_model, glove_search_phrase_vector, 300)

[array([ 3.4126e-02,  2.6860e-01,  4.7304e-01, -2.8189e-01,  4.5119e-02,
         6.2574e-01,  1.9627e-02, -5.4373e-01,  1.5018e-01,  5.5266e-01,
         3.2248e-01,  4.1528e-01,  4.6967e-01,  8.8021e-03, -1.7847e-01,
        -3.3880e-01, -1.5974e-01, -1.4349e-01,  3.4801e-01, -9.5154e-02,
        -4.8324e-01,  4.4225e-01,  1.1014e-01,  4.0039e-01, -1.6254e-01,
        -4.3039e-02,  3.8911e-01,  3.6974e-01,  1.3820e-02,  1.8999e-01,
         1.5572e-01, -4.4606e-01,  1.6935e-01, -2.4614e-01, -3.3605e-01,
         3.9507e-02, -4.6525e-01,  2.7791e-01,  7.4636e-01, -3.0076e-01,
         9.3199e-02, -6.7716e-02, -1.5919e-01,  3.0328e-02, -6.2353e-02,
         1.1201e-01,  1.7238e-01,  2.5729e-01,  3.7777e-01, -7.9195e-01,
         7.3877e-01, -6.2364e-01,  1.8464e-01, -3.9855e-02, -1.3531e-01,
         8.0921e-02, -1.2245e-01,  2.3747e-01,  8.1351e-02,  8.3093e-02,
        -1.9036e-03, -1.3692e-01,  4.1312e-01,  9.2847e-02, -7.0756e-02,
         2.8223e-01,  2.8092e-01, -1.2796e-04, -3.9

In [36]:
# Calculate cosine similarity between searched phrase and job title
glove_similarity =[]
for i in range(0, len(df)):
    sim_score = cosine_similarity(np.mean(glove_vectors[i], axis = 0), np.mean(glove_search_phrase_vector, axis =0))
    glove_similarity.append(sim_score)

# Add GloVe similarity score to the dataframe
df['GloVe_sim_score'] = glove_similarity
df.sort_values(by ='GloVe_sim_score', ascending = False).head(15)

,job_title,location,connection,nlp_job_title,cleaned_job_title,tfidf_sim_score,tfidf_sim_score2,GloVe_sim_score
5,Aspiring Human Resources Specialist,Greater New York City Area,1,aspiring human resources specialist,aspiring human resource specialist,0.629129,0.684514,0.934877
2,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,aspiring human resources professional,aspiring human resource professional,0.697367,0.774586,0.924891
45,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,aspiring human resources professional,aspiring human resource professional,0.697367,0.774586,0.924891
21,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,aspiring human resources manager seek internsh...,aspiring human resource manager seeking intern...,0.504342,0.626699,0.908395
22,Human Resources Professional,Greater Boston Area,16,human resources professional,human resource professional,0.218047,0.427058,0.871642
13,Seeking Human Resources Opportunities,"Chicago, Illinois",390,seek human resources opportunities,seeking human resource opportunity,0.145247,0.291079,0.837561
30,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,aspiring human resources professional | energe...,aspiring human resource professional energetic...,0.317168,0.382502,0.824088
12,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+,aspiring human resources management student se...,aspiring human resource management student see...,0.412685,0.437541,0.815054
49,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500+,human resources generalist loparex,human resource generalist loparex,0.129418,0.248633,0.797933
47,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,seek human resources position,seeking human resource position,0.151592,0.300910,0.785119


## Word2vec - This uses gensim model with Google News corpus

In [37]:
# Using a pre-trained word2vec model
w2v_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary = True)

In [38]:
# Vectorize job title using Word2Vec model
w2v_vectors = []
for i in range(0, len(df)):
    w2v_sentence_vector = []
    doc_token_vectors(df['cleaned_job_title'][i], w2v_model, w2v_sentence_vector, 300)
    w2v_vectors.append(w2v_sentence_vector)

In [39]:
# Vectorize search phrase using Word2Vec model
w2v_search_phrase = []
doc_token_vectors(lemmatized_phrase, w2v_model, w2v_search_phrase, 300)

[array([-1.40625000e-01,  6.83593750e-02,  1.09252930e-02, -1.72851562e-01,
         1.35742188e-01,  2.55859375e-01,  3.78417969e-02,  4.22363281e-02,
        -4.27734375e-01,  1.05468750e-01,  4.24804688e-02,  6.78710938e-02,
         1.92382812e-01,  1.25732422e-02, -1.43554688e-01,  2.73437500e-01,
         1.30859375e-01,  2.48046875e-01,  4.73632812e-02, -2.97851562e-02,
         1.32812500e-01,  2.77343750e-01, -3.18359375e-01, -1.91406250e-01,
        -4.00390625e-02, -1.38549805e-02, -2.62451172e-03,  9.03320312e-02,
         9.13085938e-02,  2.08740234e-02,  1.93359375e-01, -1.52343750e-01,
        -3.83300781e-02, -7.12890625e-02,  3.78417969e-02, -8.25195312e-02,
         1.29882812e-01, -4.85839844e-02,  1.61132812e-01, -9.96093750e-02,
         2.49023438e-02, -1.13769531e-01, -9.33837891e-03, -1.46484375e-01,
        -1.22070312e-01,  6.88476562e-02, -1.98242188e-01,  2.26562500e-01,
         8.83789062e-02,  2.78320312e-02, -1.68945312e-01,  1.44531250e-01,
         1.6

In [40]:
# Cosine Similarity score for Word2Vec model
w2v_similarity_score = []
for i in range(0, len(df)):
    sim_score = cosine_similarity(np.mean(w2v_search_phrase, axis = 0), 
                                  np.mean(w2v_vectors[i], axis = 0))
    w2v_similarity_score.append(sim_score)

# Add Word2Vec similarity score
df['Word2Vec_sim_score'] = w2v_similarity_score
df['mean_sim_score'] = df.loc[:, 'tfidf_sim_score': 'Word2Vec_sim_score'].mean(axis=1)
df.sort_values(by = 'mean_sim_score', ascending = False).head()


,job_title,location,connection,nlp_job_title,cleaned_job_title,tfidf_sim_score,tfidf_sim_score2,GloVe_sim_score,Word2Vec_sim_score,mean_sim_score
45,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,aspiring human resources professional,aspiring human resource professional,0.697367,0.774586,0.924891,0.950395,0.836810
2,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,aspiring human resources professional,aspiring human resource professional,0.697367,0.774586,0.924891,0.950395,0.836810
5,Aspiring Human Resources Specialist,Greater New York City Area,1,aspiring human resources specialist,aspiring human resource specialist,0.629129,0.684514,0.934877,0.912262,0.790195
21,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,aspiring human resources manager seek internsh...,aspiring human resource manager seeking intern...,0.504342,0.626699,0.908395,0.875945,0.728845
12,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+,aspiring human resources management student se...,aspiring human resource management student see...,0.412685,0.437541,0.815054,0.783973,0.612313


In [41]:
import fasttext
fasttext_load = fasttext.load_model("fasttext_cbow.bin")

In [48]:
len(fasttext_load.words)

18483

In [43]:
# Vectorize job title using Fasttext
fasttext_vectors = []
for i in range(0, len(df)):
    fasttext_sentence_vector = []
    doc_token_vectors(df['cleaned_job_title'][i], fasttext_load, fasttext_load_sentence_vector, 300)
    fasttext_vectors.append(fasttext_sentence_vector)



NameError: name 'fasttext_load_sentence_vector' is not defined

In [ ]:
    
# Vectorize searched phrase using Fasttext model
fasttext_search_phrase_vector = []
doc_token_vectors(lemmatized_phrase, fasttext_load, fasttext_sentence_vector, 300)
